<a href="https://colab.research.google.com/github/alakade-newageproducts/pdf-mini-project/blob/main/pdf_reader_bank_statements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tabula
import pandas as pd

# Read PDF file and extract table
df = tabula.read_pdf("/content/Last 6 Month Bank Statement.pdf", pages="all")[0]

# Extract text
pdf_json = tabula.read_pdf("/content/Last 6 Month Bank Statement.pdf", pages="all", output_format='json')
pdf_text = ""
for page in pdf_json:
    for row in page['data']:
        for col in row:
            pdf_text += col['text'] + " "

# Store table and text in a DataFrame
df = pd.DataFrame(df)
df["text"] = pdf_text

print(df.head())


Feb 20, 2023 11:31:16 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Feb 20, 2023 11:31:16 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode

Feb 20, 2023 11:31:18 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Feb 20, 2023 11:31:18 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode



       Txn Date   Value\rDate  \
0  15 Jan\r2022  15 Jan\r2022   
1  15 Jan\r2022  15 Jan\r2022   
2  16 Jan\r2022  16 Jan\r2022   
3  16 Jan\r2022  16 Jan\r2022   
4  19 Jan\r2022  19 Jan\r2022   

                                         Description  \
0          DEBIT-ACHDr\rHDFC01651000012196\rHDFCLTD-   
1    ATM WDL-ATM CASH 20151\r+THAKOOR NAGAR\rMUMBAI-   
2  TO TRANSFER-\rUPI/DR/201605324315/AKSHA\rY A/D...   
3  BY TRANSFER-\rUPI/CR/201662721675/GOOG\rLEPAY/...   
4  TO TRANSFER-\rUPI/DR/201965119219/Bank\rAcc/SB...   

             Ref No./Cheque\rNo.      Debit Credit    Balance  \
0                            NaN  11,901.00    NaN  42,999.07   
1                            NaN   1,000.00    NaN  41,999.07   
2     TRANSFER TO\r4694480162093   3,500.00    NaN  38,499.07   
3  TRANSFER\rFROM\r5098780162099        NaN   6.00  38,505.07   
4     TRANSFER TO\r5099541162096     500.00    NaN  38,005.07   

                                                text  
0  Txn Date Value\

## Extract Multiple statements and merge them into single dataframe and store it into database

In [ ]:
import tabula
import pandas as pd
import PyPDF2
import os
import sqlite3

# Set the path to the directory containing the PDF files
pdf_dir = "/content"

# Create an empty list to store the extracted tables and text
pdf_tables = []
pdf_text = ""

# Iterate over all the PDF files in the directory
for filename in os.listdir(pdf_dir):
    if filename.endswith(".pdf"):
        # Open PDF file and get number of pages
        with open(os.path.join(pdf_dir, filename), "rb") as f:
            pdf_reader = PyPDF2.PdfReader(f)
            num_pages = len(pdf_reader.pages)

        # Read PDF file and extract tables and text from all pages
        for page in range(1, num_pages + 1):
            tables = tabula.read_pdf(os.path.join(pdf_dir, filename), pages=page, pandas_options={'header': None})
            pdf_tables.extend(tables)
            pdf_json = tabula.read_pdf(os.path.join(pdf_dir, filename), pages=page, output_format='json')
            for page in pdf_json:
                for row in page['data']:
                    for col in row:
                        pdf_text += col['text'] + " "

# Concatenate tables into a single DataFrame
df = pd.concat(pdf_tables)

# Add text column to DataFrame
df["text"] = pdf_text

# Extract first row and set as column names for columns 1 to 6
cols = df.iloc[0, :6].tolist()
df = df.iloc[1:, :6]
df.columns = cols

# Add last column to DataFrame
df["text"] = pdf_text

# create a connection to the database
conn = sqlite3.connect('mydatabase.db')

# create the "bank_statement" table if it does not exist
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS bank_statement
             ([0] TEXT, [1] TEXT, [2] TEXT, [3] TEXT, [4] TEXT, [5] TEXT, [6] TEXT, text TEXT)''')

# store the DataFrame in the "bank_statement" table
df.to_sql('bank_statement', conn, if_exists='append', index=False)

# commit the changes and close the connection
conn.commit()
conn.close()

# create a connection to the database
conn = sqlite3.connect('mydatabase.db')

# retrieve the contents of the "bank_statement" table
df = pd.read_sql_query("SELECT * from bank_statement", conn)

# display the DataFrame
print(df)

# close the connection
conn.close()

df.head()


Feb 20, 2023 1:20:13 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Feb 20, 2023 1:20:13 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode

Feb 20, 2023 1:20:15 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Feb 20, 2023 1:20:15 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode

Feb 20, 2023 1:20:16 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

Feb 20, 2023 1:20:17 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

Feb 20, 2023 1:20:18 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

Feb 20, 2023 1:20:20 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

Feb 20, 2023 1:20:21 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

Feb 20, 2023 1:20:22 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

Feb 20, 2023 1:20:24 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

Feb 20, 2023 1:20:26 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

Feb 20, 2023 1:20:27 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Feb 20, 2023 1:20:27

         Txn Date   Value\rDate  \
0    15 Jan\r2022  15 Jan\r2022   
1    15 Jan\r2022  15 Jan\r2022   
2    16 Jan\r2022  16 Jan\r2022   
3    16 Jan\r2022  16 Jan\r2022   
4    19 Jan\r2022  19 Jan\r2022   
..            ...           ...   
343    5 Jul 2022    5 Jul 2022   
344    7 Jul 2022    7 Jul 2022   
345   11 Jul 2022   11 Jul 2022   
346   11 Jul 2022   11 Jul 2022   
347   12 Jul 2022   12 Jul 2022   

                                           Description  \
0            DEBIT-ACHDr\rHDFC01651000012196\rHDFCLTD-   
1      ATM WDL-ATM CASH 20151\r+THAKOOR NAGAR\rMUMBAI-   
2    TO TRANSFER-\rUPI/DR/201605324315/AKSHA\rY A/D...   
3    BY TRANSFER-\rUPI/CR/201662721675/GOOG\rLEPAY/...   
4    TO TRANSFER-\rUPI/DR/201965119219/Bank\rAcc/SB...   
..                                                 ...   
343       DEBIT-ACHDr\rICIC00261000001992 TP ACH\rHDF-   
344                 DEBIT-ACHDr 4009056\rLIC OF INDIA-   
345  BY TRANSFER-\rUPI/CR/219209984403/PRIYA\rZA/IC...   

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


,Txn Date,Value\rDate,Description,Ref No./Cheque\rNo.,Debit,Credit,text
0,15 Jan\r2022,15 Jan\r2022,DEBIT-ACHDr\rHDFC01651000012196\rHDFCLTD-,None,"11,901.00",None,Txn Date Value\rDate Description Ref No./Chequ...
1,15 Jan\r2022,15 Jan\r2022,ATM WDL-ATM CASH 20151\r+THAKOOR NAGAR\rMUMBAI-,None,"1,000.00",None,Txn Date Value\rDate Description Ref No./Chequ...
2,16 Jan\r2022,16 Jan\r2022,TO TRANSFER-\rUPI/DR/201605324315/AKSHA\rY A/D...,TRANSFER TO\r4694480162093,"3,500.00",None,Txn Date Value\rDate Description Ref No./Chequ...
3,16 Jan\r2022,16 Jan\r2022,BY TRANSFER-\rUPI/CR/201662721675/GOOG\rLEPAY/...,TRANSFER\rFROM\r5098780162099,None,6.00,Txn Date Value\rDate Description Ref No./Chequ...
4,19 Jan\r2022,19 Jan\r2022,TO TRANSFER-\rUPI/DR/201965119219/Bank\rAcc/SB...,TRANSFER TO\r5099541162096,500.00,None,Txn Date Value\rDate Description Ref No./Chequ...
